# A python parser for the BRENDA database

This project provides python classes and functions to parse the text file containing the entire BRENDA enzyme database (https://www.brenda-enzymes.org)

This is an ongoing project!

In [1]:
from parseBRENDA import BRENDA
workDir = 'C:/Users/tinta/OneDrive/Documents/Projects/BRENDA'
dataFile = workDir + '/brenda_download.txt'

In [2]:
# Let's load the database
brenda = BRENDA(dataFile)
brenda

Number of Enzymes,7558
BRENDA copyright,"Copyrighted by Dietmar Schomburg, Techn. University Braunschweig, GERMANY. Distributed under the License as stated at http:/www.brenda-enzymes.org"
Parser version,0.1
Author,"Semidán Robaina Estévez, 2020"


In [6]:
# We can retrieve an enzyme entry by its EC number like this
r = brenda.reactions.get_by_id('2.7.1.40')
r

Enzyme identifier,2.7.1.40
Name,pyruvate kinase
Systematic name,ATP:pyruvate 2-O-phosphotransferase
Reaction type,phospho group transfer
Mechanism,"ATP + pyruvate <=> ADP + phosphoenolpyruvate (#6,23,32,49,69,70,92#mechanism <5,92>; #69# compulsory-ordered tri-bi mechanism <100>; #103#model for allosteric regulation <91>; #120,121# allosteric enzyme <98>;#16,18,36# allosteric enzyme: homotropic <73,74,75>; #12# hyperbolickinetics <96>; #118# sigmoidal kinetics with respect tophosphoenolpyruvate <93>; #52# catalyzes the addition of a proton andthe loss of a phosphoryl group which is transferred to ADP <48>; #98#sigmoidal saturation curves with substrate and metal ions <89>; #140#modeling of the catalytic mechanism, overview <248>; #149# the kineticmechanism is random order with a rapid equilibrium <255>)"


In [11]:
# Here are all the KM values associated with this enzyme
r.getKMvalues().filter_by_organism('Bos taurus')['ADP'][0]['value']

0.275

In [12]:
# We can also get information about operational temperatures of the enzyme
temp = r.getTemperatureData()

In [18]:
print([l['value'] for l in temp['optimum']])

[30.0, 37.0, 95.0, 22.0, 23.0, 44.0, 28.0, 40.0, 55.0, 45.0, -999.0, 60.0, 15.0, 20.5, 85.0, 50.0, 80.0, 25.0, 98.0]


In [20]:
print([l['value'] for l in temp['range']][0])

[35.0, 53.0]


In [25]:
r.getKcatvalues()['phosphoenolpyruvate'][0]

{'value': 3.2,
 'species': {'Homo sapiens'},
 'meta': '#10# wild type enzyme, in the absenceof K+, in 50 mM Mes-Tris, pH 6.0, at 25Â°C <142>',
 'refs': ['142']}

In [22]:
r.getSubstratesAndProducts()    

[{'substrates': ['AKT1S1', 'ATP'], 'products': ['ADP', 'phospho-AKT1S1']},
 {'substrates': ['TDP', 'phosphoenolpyruvate'],
  'products': ['TTP', 'pyruvate | 95% yield |']},
 {'substrates': ['ATP', 'pyruvate'],
  'products': ['ADP', 'phosphoenolpyruvate']},
 {'substrates': ['ADP', 'phosphoenolpyruvate'],
  'products': ['ATP', 'pyruvate']},
 {'substrates': ['ATP', 'prothymosin alpha'],
  'products': ['ADP', 'phospho-prothymosin alpha']}]